# JEM092 - Asset Pricing HW3 - Pavlína Křenková & Tomáš Svoboda

In [1]:
# libraries
shhh = suppressPackageStartupMessages
shhh(library(quantmod))
shhh(library(sandwich))
shhh(library(xts))
shhh(library(lubridate))
shhh(library(moments))

# 1. Introduction

This paper uses independent bivariate portfolio analysis and the Fama-MacBeth regression model to assess the cross-sectional relation between future stock returns and variables: size, volatility, and skewness. The bivariate portfolios are created as *size-volatility* and *size-skewness*. During interpretation, emphasis is given to the effect of volatility and skewness, and how controlling for size affects the results. 

First, we load all the necessary data from external sources and previous home assignments. Then, we split the data according to the sorting variables into 3x3 portfolios, using the 30th and 70th percentiles as breakpoints. Then we prepare the code for the calculations and present the results. The paper aims to evaluate the cross-sectional relations between the sort variables and the returns. We create the difference long-short portfolios. We discuss whether our results align with theory and confirm the assumptions, or whether we observe any major differences. 

# 2. Data

The data needed for this analysis are: daily adjusted closing prices of the individual stocks, market capitalization, market risk premium (MKT), the risk-free rate, small-minus-big factor (SMB), high-minus-low factor (HML), momentum factor (MOM) and the liquidity factor (LIQ).

The underlying adjusted closing prices and market capitalizations were downloaded in the previous homework and we load them here. Other needed variables, namely the market risk premium, small-minus-big factor, high-minus-low factor, and momentum factors are downloaded from the http://mba.tuck.dartmouth.edu and then loaded in the notebook. Moreover, the liquidity factor is needed, which is downloaded from https://faculty.chicagobooth.edu. The loading process is done for each series individually due to the unique structure of the data.

As mentioned, this paper concerns bivariate sorts, where the sort variables are size and two additional variables. We were assigned variables *volatility* and *skewness*, hence no more data is downloaded since these are calculated directly from the returns. 

The data preparation process is mostly analogous to our HW2 with only a few differences (the book-to-market ratio is not needed, while skewnesses need to be calculated).

In [2]:
# See assigned factors
factors_all = read.csv('second_third_factor_rand.csv')
head(factors_all)
factors_all[factors_all$Group.Number == '98698413',] # 1st assigned Factor (Pavlina's number)
factors_all[factors_all$Group.Number == '60115410',] # 2nd assigned Factor (Tomas' number)

,X,Group.Number,Assigned.Factor
,<int>,<int>,<chr>
1,1,36654797,MOM
2,2,40354250,VOL
3,3,78853749,SKEW
4,4,62727848,SKEW
5,5,44081550,STR
6,6,31690322,ILLIQ


,X,Group.Number,Assigned.Factor
,<int>,<int>,<chr>
18,18,98698413,SKEW


,X,Group.Number,Assigned.Factor
,<int>,<int>,<chr>
42,42,60115410,VOL


## a) Data Load

In [3]:
# Load data from HW1
price_volume_xts <- readRDS("price_volume_xts.rds")
market_cap_xts <- readRDS("market_cap_xts.rds") # Lists of xts objects
lapply(price_volume_xts, head)[1]

$UDR
           UDR.Adjusted UDR.Volume
2010-01-04     9.739022    1835200
2010-01-05     9.727006    1665600
2010-01-06     9.660918    1889000
2010-01-07     9.733015    1792000
2010-01-08     9.540755    1099800
2010-01-11     9.606844    1531200


In [4]:
# Download needed factors
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
# Daily FF5 factors - used for daily excess returns
download.file(
    'http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_TXT.zip',
    destfile = 'F-F_Research_Data_5_Factors_2x3_daily_TXT.zip'
)
# Momentum
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_TXT.zip",
    destfile = "F-F_Momentum_Factor.zip"
)
unzip("F-F_Research_Data_Factors.zip")
unzip('F-F_Research_Data_5_Factors_2x3_daily_TXT.zip')
unzip("F-F_Momentum_Factor.zip")

Load monthly FF3 factors

In [5]:
# Load factors - monthly
FF3factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1170,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
head(FF3factors)
tail(FF3factors)

,t,mkt.rf,smb,hml,rf
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,192607,2.96,-2.56,-2.43,0.22
2,192608,2.64,-1.17,3.82,0.25
3,192609,0.36,-1.40,0.13,0.23
4,192610,-3.24,-0.09,0.70,0.32
5,192611,2.53,-0.10,-0.51,0.31
6,192612,2.62,-0.03,-0.05,0.28


,t,mkt.rf,smb,hml,rf
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1165,202307,3.21,2.08,4.11,0.45
1166,202308,-2.39,-3.16,-1.06,0.45
1167,202309,-5.24,-2.51,1.52,0.43
1168,202310,-3.19,-3.88,0.18,0.47
1169,202311,8.84,-0.02,1.64,0.44
1170,202312,4.87,6.34,4.93,0.43


In [6]:
FF3factors$t <- as.Date(as.yearmon(as.character(FF3factors$t), format = "%Y%m"), frac = 1) # transform to valid time
FF3factors_xts <- xts(FF3factors[, -1], order.by = FF3factors$t)
head(FF3factors)

,t,mkt.rf,smb,hml,rf
,<date>,<dbl>,<dbl>,<dbl>,<dbl>
1,1926-07-31,2.96,-2.56,-2.43,0.22
2,1926-08-31,2.64,-1.17,3.82,0.25
3,1926-09-30,0.36,-1.40,0.13,0.23
4,1926-10-31,-3.24,-0.09,0.70,0.32
5,1926-11-30,2.53,-0.10,-0.51,0.31
6,1926-12-31,2.62,-0.03,-0.05,0.28


Load daily FF factors

In [7]:
# Daily 5 factors data -> used for calculating daily excess returns - contains daily rf
FF5_factors <- read.delim(
    'F-F_Research_Data_5_Factors_2x3_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rmw', 'cma', 'rf'),
    sep = '',
    nrows = 25438,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
FF5_factors$t <- as.Date(as.character(FF5_factors$t), format='%Y%m%d')
FF5_factors_xts <- as.xts(FF5_factors[, 2:7], order.by = FF5_factors$t)
head(FF5_factors)

,t,mkt_rf,smb,hml,rmw,cma,rf
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012
5,1963-07-09,0.45,0.00,0.09,0.15,-0.01,0.012
6,1963-07-10,-0.18,0.20,0.00,0.05,-0.09,0.012


Momentum

In [8]:
# Momentum
FFmom <- read.delim(
    'F-F_Momentum_Factor.txt',
    col.names = c('t', 'mom'), 
    sep = '',
    nrows = 1164,
    header = TRUE,
    skip = 13,
    stringsAsFactors = FALSE
)
# Set date as index
FFmom$t <- as.Date(as.yearmon(as.character(FFmom$t), format = "%Y%m"), frac = 1)
FFmom_xts <- xts(FFmom[, -1], order.by = FFmom$t)
colnames(FFmom_xts) <- "mom"
tail(FFmom)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“header and 'col.names' are of different lengths”


,t,mom
,<date>,<dbl>
1159,2023-07-31,-3.98
1160,2023-08-31,3.77
1161,2023-09-30,0.26
1162,2023-10-31,1.72
1163,2023-11-30,2.75
1164,2023-12-31,-5.51


Liquidity

In [9]:
# Liquidity
data_url <- "https://faculty.chicagobooth.edu/-/media/faculty/lubos-pastor/research/liq_data_1962_2023.txt"

# Reading data directly
liq_factor <- read.delim(
    file = url(data_url),
    col.names = c('Month', 'Agg Liq.', 'Innov Liq (eq8)', 'Traded Liq (LIQ_V)'), 
    sep = '',
    header = TRUE,
    skip = 10,
    stringsAsFactors = FALSE
)

# Processing dates
liq_factor$Month <- as.Date(zoo::as.yearmon(as.character(liq_factor$Month), format = "%Y%m"), frac = 1)
liq_factor_xts <- xts::xts(liq_factor[, -1], order.by = liq_factor$Month)

# Viewing the last entries
tail(liq_factor_xts)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“header and 'col.names' are of different lengths”


              Agg.Liq. Innov.Liq..eq8. Traded.Liq..LIQ_V.
2023-07-31  0.03158360      0.05313238        -0.03521294
2023-08-31 -0.02219315     -0.01384346        -0.04206628
2023-09-30 -0.06412711     -0.03281943        -0.04053745
2023-10-31  0.16340711      0.17621454         0.00078403
2023-11-30  0.01836070     -0.10736516         0.05513213
2023-12-31 -0.09925067     -0.12738569         0.03728561

## b) Data prep

We unify the time interval of each data frame to be between 2010 and 2023, following HW2 with the same restriction. Then we convert the data to the same format for easier handling.

In [10]:
start_date <- as.Date("2010-01-01")
end_date <- as.Date("2023-12-31")

adjust_sample_period <- function(xts_series) {
    lapply(xts_series, function(stock){
        stock[paste(start_date, "/", end_date, sep = "")]
    })
}

price_volume <- adjust_sample_period(price_volume_xts)
market_cap <- adjust_sample_period(market_cap_xts)
three_factors <- adjust_sample_period(FF3factors_xts)
five_factors <- adjust_sample_period(FF5_factors_xts)
momentum_factor <- adjust_sample_period(FFmom_xts)
liquidity_factor <- adjust_sample_period(liq_factor_xts)

In [11]:
# Extract the values
price_list <- lapply(price_volume, function(x) x[,1])
                     
# Convert lists to dataframes
prices <- do.call('cbind', price_list)
nominals <- do.call('cbind', market_cap)

three_fact <- do.call('cbind', three_factors)
five_fact <- do.call('cbind', five_factors)
momentum <- momentum_factor$mom
liquidity <- liquidity_factor$Traded.Liq..LIQ_V. # Traded liquidity factor (LIQ_V)

# Set column names
colnames(prices) <- names(price_list)
colnames(nominals) <- names(market_cap)

colnames(liquidity) <- "liquidity"

## c) Monthly returns, volatilities, skewnesses and market capitalizations  

We calculate the excess returns as $excess.returns_{i,t} = returns_{i,t} - risk.free.rate_{t}$, which will be used in further analysis. The approach is generally consistent with HW2; moreover, skewness is calculated. The monthly volatilities and skewnesses are calculated from the daily data we obtained previously. The volatility is annualized and scaled to be in percentages. Monthly nominals/market capitalizations are also calculated. 

In [12]:
# Monthly & daily returns
daily_returns <- lapply(prices, function(x) suppressWarnings(dailyReturn(x)))
monthly_returns <- lapply(prices, function(x) suppressWarnings(quantmod::monthlyReturn(x)))
daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <-names(price_list)
colnames(monthly_returns) <- names(price_list)
index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns), format = "Q%q/%y"), frac = 1)

# Excess returns
for (i in 1:ncol(monthly_returns)) {
    daily_returns[, i] <- daily_returns[, i] - five_fact[, "rf"]
    monthly_returns[, i] <- monthly_returns[, i] - three_fact[, "rf"]
}

# Monthly volatilities
monthly_volatilities <- as.xts(aggregate(daily_returns, as.yearmon(time(daily_returns)), sd))*100*sqrt(12) # Extract time index from daily_returns, convert it to 'yearmon' format, then aggregate/group daily returns with month times, then calculates sd within each month, then convert to xts object
index(monthly_volatilities) <- as.Date(as.yearmon(index(monthly_volatilities), format = "Q%q/%y"), frac = 1)
colnames(monthly_volatilities) <- names(price_list)

# Monthly skewnesses
monthly_skewness <- as.xts(aggregate(daily_returns, as.yearmon(time(daily_returns)), FUN = skewness)) # Extract time index from daily_returns, convert it to 'yearmon' format, then aggregate/group daily returns with month times, then calculates skewness within each month, then convert to xts object
index(monthly_skewness) <- as.Date(as.yearmon(index(monthly_skewness)), frac = 1)
colnames(monthly_skewness) <- names(price_list)


# Monthly nominals
monthly_nominals <- as.xts(aggregate(nominals, as.yearmon(time(nominals)), sum))
index(monthly_nominals) <- as.Date(as.yearmon(index(monthly_nominals), format = "Q%q/%y"), frac = 1)
colnames(monthly_nominals) <- names(price_list)

Here we calculate the standard deviations of the volatility and skewness, which are then used to discuss economic importance of the factors. The SD is calculated for each mon of the cross-section, and then the time series mean is calculated.

In [13]:
# used later to determine economic importance
monthly_sd = apply(na.omit(monthly_volatilities), 1, sd)
mean_monthly_sd_vol = mean(monthly_sd)
mean_monthly_sd_vol

monthly_sd = apply(na.omit(monthly_skewness), 1, sd)
mean_monthly_sd_skew = mean(monthly_sd)
mean_monthly_sd_skew


[1] 2.462899

[1] 0.8180215

## d) Sizes

The sizes are calculated as $$size_{i,t} = ln(market.cap_{i,t})$$

In [14]:
monthly_sizes <- log(monthly_nominals)
colnames(monthly_sizes) <- names(price_list)

# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))

## e) Overview

All FF-CPS factors we will be using are merged into the *factors* dataframe, the final montly data for returns, volatilities, skewnesses, nominals and sizes are prepared in their respective dataframes.

In [15]:
factors <- cbind(three_fact, momentum, liquidity)

In [16]:
head(factors)

           mkt.rf   smb   hml   rf   mom   liquidity
2010-01-31  -3.36  0.40  0.43 0.00 -5.40 -0.01105698
2010-02-28   3.40  1.19  3.23 0.00  3.74 -0.00755895
2010-03-31   6.31  1.48  2.21 0.01  3.76 -0.06257515
2010-04-30   2.00  4.87  2.89 0.01  3.16  0.01020015
2010-05-31  -7.89  0.09 -2.44 0.01 -0.25 -0.02806075
2010-06-30  -5.57 -1.82 -4.70 0.01 -2.76  0.00383754

In [17]:
monthly_returns[1:5, 1:3]
monthly_volatilities[1:5, 1:3]
monthly_skewness[21:25, 1:3]
monthly_nominals[1:5, 1:3]
monthly_sizes[1:5, 1:3]

                    UDR          CPB         MTB
2010-01-31 -0.028893357 -0.022438759  0.10058214
2010-02-28  0.079691093  0.006644808  0.05950287
2010-03-31  0.039999893  0.058855367  0.01518418
2010-04-30  0.152766732  0.004427361  0.09040268
2010-05-31 -0.009015349 -0.011394121 -0.09486931

                 UDR      CPB      MTB
2010-01-31  5.738571 3.371380 8.528075
2010-02-28  6.842704 3.864216 6.891594
2010-03-31  3.819788 2.454574 4.250656
2010-04-30  8.196885 2.037019 6.960964
2010-05-31 10.892669 4.374078 9.999670

                   UDR        CPB        MTB
2011-09-30 -0.45863296 -0.1595637  0.6499374
2011-10-31 -0.07348823  0.3209585 -0.2794157
2011-11-30 -0.07251836 -1.1003162  0.3868717
2011-12-31  0.19692047  0.1935681  0.5754282
2012-01-31 -0.18128541 -0.3488545 -0.5611113

             UDR    CPB    MTB
2010-01-31 46.58 214.98 164.96
2010-02-28 47.10 214.49 166.65
2010-03-31 62.52 270.62 217.82
2010-04-30 66.23 253.27 211.25
2010-05-31 65.74 241.72 200.58

                UDR      CPB      MTB
2010-01-31 3.841171 5.370545 5.105703
2010-02-28 3.852273 5.368263 5.115896
2010-03-31 4.135487 5.600716 5.383669
2010-04-30 4.193134 5.534456 5.353042
2010-05-31 4.185708 5.487780 5.301213

# 3. Methodology

In this section, we prepare the calculation of the bivariate sort and the Fama-MacBeth regression. 

First, we perform the bivariate independent sort based on pairs *size-volatility* and *size-skewness*. Both equal- and value-weighting schemes are prepared. Difference portfolios are prepared and the average next-month excess returns are calculated for each portfolio, as well as the alphas of the Fama-French-Carhart-Pastor-Stambaugh model and the CAPM model. T-statistics are prepared using the Newey-West method, which makes inference valid even in the case of autocorrelation or heteroskedasticity.

Finally, the Fama-MacBeth regression is estimated. The code allows for the estimation of the regression both for the individual factors as well as for combinations of the factors.

## a) Bivariate portfolio analysis

The bivariate sort is based on an independent sort analysis, where the 30th and 70th percentiles are the breakpoints. This means that the breakpoints are calculated independently of the each other, not taking the breakpoints of the other variable into account, and the order of the sort variables thus does not matter. 

The result is 9 base portfolios - all combinations of the 3 size portfolios with the 3 portfolios of the 2nd sort variables, and 6 difference portfolios - three for each sort variable.

This is performed in the function *`portfolio_returns`*. To iterate through all the 9 portfolios we use nested for-loops, where the outer for-loop loops through the sort variable 1 (in our case *size*, index j), while the inner for-loop through the 2nd variable (*volatility*/*skewness*, index k). The difference portfolios are hence based on this observation: e.g. the '6-4' portfolio is the difference portfolio of the 2nd sort variable, in the 2nd portfolio of the 1st sort variable, etc. This can be observed from the table below, which corresponds to the indices from the code.

| Portfolio |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |
|-----------|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| **j, k**  |(1,1)|(1,2)|(1,3)|(2,1)|(2,2)|(2,3)|(3,1)|(3,2)|(3,3)|

When the portfolios are created accordingly, the returns are averaged either with equal weights or weights proportional to company market capitalization (value-weighting). The function expects 2 data sets, 1 for each sort variable, and a specified weighting type. Monthly excess returns are passed and for each portfolio the one-month-ahead average excess return is calculated. The monthly market capitalisations are used for the value weighting.

In [18]:
portfolio_returns <- function(monthly_sort_variable1, monthly_sort_variable2, type, num_portfolios=3, monthly_ret=monthly_returns, monthly_nom=monthly_nominals) {
    n_portfolios <- num_portfolios*num_portfolios # number of portfolios
    diff_portfolio <- c('3-1', '6-4', '9-7', '7-1', '8-2', '9-3') # long-short portfolios

    # Initialize an xts object for storing portfolio returns.
    portfolio_ret <- as.xts( 
        matrix(
            nrow = nrow(monthly_sort_variable1) - 1,
            ncol = n_portfolios + length(diff_portfolio)  # Last one for Diff portfolio
        ),
        order.by = index(monthly_sort_variable1[2:nrow(monthly_sort_variable1)]) # Order indexes
    )
    names(portfolio_ret) <- c(1:n_portfolios, diff_portfolio) # Add column names

    # Iterate over rows, find breakpoints and compute month returns within the given nominal bounds.
    for (i in 1:nrow(portfolio_ret)) { # For every month
        current_month <- as.Date(format(index(portfolio_ret[i]), format="%Y-%m-%d"))
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(monthly_ret)) {
            next
        }
        if (type == "VW") { # otherwise the missing market cap does not matter
            if (!next_month %in% index(monthly_nom)) {
                next
            }        
        }

        # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
        month_returns <- monthly_ret[next_month] # Returns in next month
        month_market_caps <- monthly_nom[next_month] # Market caps in next month

        # Replace NA market caps with 0s so that such observations have no weights.
        month_market_caps[is.na(month_market_caps)] <- 0
        month_sort_variable1 <- monthly_sort_variable1[current_month] # Sort variable for the respective month
        month_sort_variable2 <- monthly_sort_variable2[current_month] 

        if (ncol(month_sort_variable1) == sum(is.na(month_sort_variable1))) {
            next
        }
        # It may happen that we have empty row, which is not equal to have NA rows
        if (ncol(month_sort_variable2) == 0 || nrow(month_sort_variable2) == 0) {
        next
        }
        
        # Breakpoints        
        breakpoints_var1 <- quantile(month_sort_variable1, c(0, 0.3, 0.7, 1), na.rm = TRUE)
        breakpoints_var1[1] <- breakpoints_var1[1] - 1
        breakpoints_var1[length(breakpoints_var1)] <- breakpoints_var1[length(breakpoints_var1)] + 1
        not_na_var1 <- !is.na(month_sort_variable1)
        # Independently filter for Size
        breakpoints_var2 <- quantile(month_sort_variable2, c(0, 0.3, 0.7, 1), na.rm = TRUE)
        breakpoints_var2[1] <- breakpoints_var2[1] - 1
        breakpoints_var2[length(breakpoints_var2)] <- breakpoints_var2[length(breakpoints_var2)] + 1
        not_na_var2 <- !is.na(month_sort_variable2)

        for (j in 1:num_portfolios) { # For every portfolio
            filter_var1 <- (
                (breakpoints_var1[[j]] <= month_sort_variable1)
                & (month_sort_variable1 < breakpoints_var1[[j + 1]])
                & not_na_var1
            )
            filter_var1 = data.frame(coredata(filter_var1))
            for (k in 1:num_portfolios) {
                filter_var2 <- (
                    (breakpoints_var2[[k]] <= month_sort_variable2)
                    & (month_sort_variable2 < breakpoints_var2[[k + 1]])
                    & not_na_var2
                )
                filter_var2 = data.frame(coredata(filter_var2))
                filtered_next_month_returns <- month_returns[, filter_var1 & filter_var2]
                filtered_month_market_caps <- month_market_caps[, filter_var1 & filter_var2]
            # Find stocks that have market cap between the breakpoints and are not NA 
                if (type == "VW") {
                    portfolio_ret[i, k + (j - 1) * num_portfolios] <- weighted.mean(
                        t(filtered_next_month_returns),
                        t(filtered_month_market_caps), na.rm = TRUE) # Calculate the mean return of the stocks in a respective portfolio
                } else if (type == "EW") {
                    portfolio_ret[i, k + (j - 1) * num_portfolios] <- mean(filtered_next_month_returns, na.rm = TRUE)
                } else {
                    stop("Incorrect portfolio-weighting method")
                }
        }
    }
    for (diff in diff_portfolio) {
        # Parse the diff string to find out which columns to subtract
        indices <- as.numeric(strsplit(diff, '-')[[1]])
        if (length(indices) == 2) {
            if (all(indices <= ncol(portfolio_ret))) {
                portfolio_ret[i, diff] <- portfolio_ret[i, indices[1]] - portfolio_ret[i, indices[2]]
            }}
        } 
    }

    return (portfolio_ret)
}

The results are average (EW/VW) monthly one month ahead excess returns. We create two independent equally weighted bivariate sorts and two value-weighted: size/volatility and size/skewness for each weighting scheme. An example of the output can be seen below.

#### (i) Equally-weighted portfolio

In [19]:
# Mean returns for each month per equally-weighted portfolio
EW_portfolio_returns_size_vol = portfolio_returns(monthly_sizes, monthly_volatilities, type="EW")
EW_portfolio_returns_size_skew = portfolio_returns(monthly_sizes, monthly_skewness, type="EW")

print("EW size and volatility example: ")
head(EW_portfolio_returns_size_vol)

print("EW size and skewness example: ")
EW_portfolio_returns_size_skew[31:36,]

[1] "EW size and volatility example: "


                     1           2           3           4           5
2010-02-28  0.05851760  0.05579364  0.09367208  0.03350870  0.06271577
2010-03-31  0.03070520  0.03280177  0.04029818  0.04078212  0.03160901
2010-04-30 -0.08502417 -0.05257986 -0.06541818 -0.05446137 -0.05987476
2010-05-31 -0.02416539 -0.03359947 -0.10710484 -0.03549397 -0.04771205
2010-06-30  0.05011672  0.05188104  0.06437388  0.02135398  0.06432667
2010-07-31 -0.01696202 -0.05914742 -0.07480750 -0.02565694 -0.05583016
                     6           7           8           9          3-1
2010-02-28  0.06999803  0.04132853  0.05707162  0.05754951  0.035154480
2010-03-31  0.04109984  0.01842276  0.02355312 -0.01117631  0.009592981
2010-04-30 -0.10021853 -0.08177637 -0.08884142 -0.12419828  0.019605984
2010-05-31 -0.07687306 -0.01838055 -0.07847880 -0.06698075 -0.082939451
2010-06-30  0.10290124  0.03208881  0.05078699  0.10024384  0.014257167
2010-07-31 -0.04849929 -0.02519839 -0.07897220 -0.06784144 -0.057845478

[1] "EW size and skewness example: "


                      1           2             3            4           5
2012-08-31  0.006617409 0.020380758  0.0240373313  0.013330401  0.02577724
2012-09-30 -0.022316717 0.001620853 -0.0008095746 -0.022238926 -0.02370710
2012-10-31  0.033121343 0.012194453  0.0123639295  0.027400471  0.01658840
2012-11-30  0.004455067 0.012024229  0.0101474415  0.013208929  0.01083653
2012-12-31  0.093321253 0.086032718  0.0705348088  0.058470025  0.06159053
2013-01-31  0.054683198 0.021997544  0.0198002759  0.009445739  0.01509931
                      6            7            8             9          3-1
2012-08-31  0.005744949  0.002476961 -0.003816420  0.0262781860  0.017419923
2012-09-30 -0.016697293 -0.011307165 -0.007060813 -0.0157723930  0.021507142
2012-10-31 -0.003060904  0.001062034  0.001125969 -0.0006951077 -0.020757413
2012-11-30  0.004880756  0.021181070  0.008812903 -0.0104227449  0.005692374
2012-12-31  0.063620329  0.058257541  0.071481706  0.0507132200 -0.022786444
2013-01-31  0

#### (ii) Value-weighted portfolio

In [20]:
# Mean returns for each month per value-weighted portfolio
VW_portfolio_returns_size_vol = portfolio_returns(monthly_sizes, monthly_volatilities, type="VW")
VW_portfolio_returns_size_skew = portfolio_returns(monthly_sizes, monthly_skewness, type="VW")

print("VW size and volatility example: ")
head(VW_portfolio_returns_size_skew)

print("VW size and skewness example: ")
VW_portfolio_returns_size_skew[31:36,]

[1] "VW size and volatility example: "


                     1           2           3           4           5
2010-02-28  0.06971537  0.08456039  0.04187677  0.05904144  0.05082554
2010-03-31  0.03970187  0.03825496  0.02386638  0.04257751  0.03180116
2010-04-30 -0.07406118 -0.05390701 -0.06587224 -0.06866628 -0.07325177
2010-05-31 -0.04893724 -0.04929834 -0.08610258 -0.03056132 -0.05818627
2010-06-30  0.07622587  0.05614670  0.05053646  0.06280773  0.06282712
2010-07-31 -0.03138223 -0.05778875 -0.06644011 -0.05504624 -0.02248277
                     6            7           8           9          3-1
2010-02-28  0.06911684  0.062183676  0.05703043  0.04743123 -0.027838605
2010-03-31  0.04026674  0.006638947  0.01559804  0.02437666 -0.015835485
2010-04-30 -0.07106461 -0.114070670 -0.10414186 -0.07707597  0.008188937
2010-05-31 -0.05761764 -0.023473222 -0.08193604 -0.06305043 -0.037165336
2010-06-30  0.07257050  0.042001735  0.06089141  0.06873392 -0.025689413
2010-07-31 -0.05502673 -0.079748659 -0.07345738 -0.04113476 -0.03

[1] "VW size and skewness example: "


                       1            2            3           4            5
2012-08-31  0.0066415489  0.006435532  0.025816794  0.01361620  0.027134551
2012-09-30 -0.0226082552 -0.001962962 -0.008153928 -0.01775702 -0.022618931
2012-10-31  0.0413792357  0.013468197  0.005658642  0.02501002  0.016062312
2012-11-30  0.0006030526  0.013062321  0.015214700  0.01335440  0.009340076
2012-12-31  0.0924861155  0.088558834  0.060372347  0.06320427  0.063431309
2013-01-31  0.0506443180  0.025218867  0.022035377  0.01131614  0.019124385
                      6             7            8            9         3-1
2012-08-31  0.006343079  0.0104520157 -0.003823807  0.044255308  0.01917525
2012-09-30 -0.017549501 -0.0490602682 -0.001162341 -0.023271321  0.01445433
2012-10-31 -0.001171484  0.0002140192 -0.003420078 -0.009804994 -0.03572059
2012-11-30  0.005798263  0.0142811945 -0.001508066 -0.035966287  0.01461165
2012-12-31  0.064406465  0.0145653524  0.057750867  0.050375500 -0.03211377
2013-01-31  

In [21]:
colMeans(na.omit(EW_portfolio_returns_size_vol))

1             2             3             4             5 
-0.0557184868 -0.0539594142 -0.0478882994 -0.0616677313 -0.0578515960 
            6             7             8             9           3-1 
-0.0568584746 -0.0601356808 -0.0610934110 -0.0609370140  0.0078301874 
          6-4           9-7           7-1           8-2           9-3 
 0.0048092567 -0.0008013332 -0.0044171940 -0.0071339969 -0.0130487146

## b) Average values

The function `portfolio_results` serves for calculating the final table. It accepts the previously calculated returns shown above and a table of average returns for each portfolio is calculated. For the difference portfolios, the Newey West t-statistics are also calculated. The estimated CAPM alpha and Fama-French-Carhart-Pastor-Stambaugh alpha and their respective Newey West t-statistics are also reported. The mean return is obtained by doing a regression only with the intercept.

To make the table readable, the function expects a string specifying the sort variables, used for row and column names.

The two alphas come from the estimated following models, using the previously prepared factors, for portfolio p:

FF-CPS alpha: $r_{p, t}=\alpha_p + \beta_{MKT, p} MKT_t + \beta_{SMB, p} SMB_t + \beta_{HML, p} HML_t + \beta_{MOM, p} MOM_t + \beta_{LIQ, p} LIQ_t + \epsilon_{p, t}$

CAPM alpha: $r_{p, t}=\alpha_p + \beta_{MKT, p} MKT_t + \epsilon_{p, t}$

In [22]:
portfolio_results <- function(portfolio_returns_df, num_portfolios=3, fact=factors, sort_var1 = 'size', sort_var2) { # sort_var1/2 needed for row/col names
    n_portfolios <- num_portfolios*num_portfolios
    results <- as.data.frame(
        matrix(nrow = num_portfolios + 6, ncol = num_portfolios + 6),
        # name columns and rows including the variable name for readability
        row.names = c(
            paste(sort_var2, 1:num_portfolios),
            paste(sort_var2, num_portfolios, '- 1'),
            paste(sort_var2, num_portfolios, '- 1 t-stat'),
            paste(sort_var2, num_portfolios, '- 1 CAPM alpha'),
            paste(sort_var2, num_portfolios, '- 1 CAPM alpha t-stat'),
            paste(sort_var2, num_portfolios, '- 1 FF-CPS alpha'),
            paste(sort_var2, num_portfolios, '- 1 FF-CPS alpha t-stat')
        ))
    names(results) <- c(
            paste(sort_var1,1:num_portfolios),
            paste(sort_var1, num_portfolios, '- 1'),
            paste(sort_var1, num_portfolios, '- 1 t-stat'),
            paste(sort_var1, num_portfolios, '- 1 CAPM alpha'),
            paste(sort_var1, num_portfolios, '- 1 CAPM alpha t-stat'),
            paste(sort_var1, num_portfolios, '- 1 FF-CPS alpha'),
            paste(sort_var1, num_portfolios, '- 1 FF-CPS alpha t-stat')
        )
    # setting values to main table + difference portfolios and alphas for 2nd sort var
    for (j in 1:num_portfolios) {
        for (k in 1:num_portfolios) {
            model <- lm(na.omit(portfolio_returns_df[, k + (j - 1) * num_portfolios]) ~ 1)
            results[k, j] <- model$coefficients[[1]]
        }
        returns_diff <- portfolio_returns_df[, n_portfolios + j]
        model_diff <- lm(na.omit(returns_diff) ~ 1)
        results[num_portfolios + 1, j] <- model_diff$coefficients[[1]]
        results[num_portfolios + 2, j] <- model_diff$coefficients[[1]] / sqrt(NeweyWest(model_diff, lag = 6)[[1]])
        is_not_na_filter = rowSums(is.na(returns_diff)) == 0
        dates = index(returns_diff[is_not_na_filter])
        dates = as.Date(format(dates), format="%Y-%m-%d")
        model_diff_ff <- lm(
            returns_diff[is_not_na_filter] ~ (
                1
                + fact[dates]$mkt.rf
                + fact[dates]$smb
                + fact[dates]$hml
                + fact[dates]$mom
                + fact[dates]$liquidity
            ))
        model_diff_capm <- lm(
            returns_diff[is_not_na_filter] ~ (
                1
                + fact[dates]$mkt.rf
            ))
        results[num_portfolios + 3, j] <- model_diff_capm$coefficients[[1]]
        results[num_portfolios + 4, j] <- (
            model_diff_capm$coefficients[[1]] / sqrt(NeweyWest(model_diff_capm, lag = 6)[[1]]))
        results[num_portfolios + 5, j] <- model_diff_ff$coefficients[[1]]
        results[num_portfolios + 6, j] <- (
            model_diff_ff$coefficients[[1]] / sqrt(NeweyWest(model_diff_ff, lag = 6)[[1]])
        )
    }
     # setting values to main table + difference portfolios and alphas for 1st sort var
    for (j in 1:num_portfolios) {
        returns_diff <- portfolio_returns_df[, n_portfolios + j + num_portfolios]
        model_diff <- lm(na.omit(returns_diff) ~ 1)
        results[j, num_portfolios + 1] <- model_diff$coefficients[[1]]
        results[j, num_portfolios + 2] <- model_diff$coefficients[[1]] / sqrt(NeweyWest(model_diff, lag = 6)[[1]])
        is_not_na_filter = rowSums(is.na(returns_diff)) == 0
        dates = index(returns_diff[is_not_na_filter])
        dates = as.Date(format(dates), format="%Y-%m-%d")
        model_diff_ff <- lm(
            returns_diff[is_not_na_filter] ~ (
                1
                + fact[dates]$mkt.rf
                + fact[dates]$smb
                + fact[dates]$hml
                + fact[dates]$mom
                + fact[dates]$liquidity
            )
        )
        model_diff_capm <- lm(
            returns_diff[is_not_na_filter] ~ (
                1
                + fact[dates]$mkt.rf
            ))
        results[j, num_portfolios + 3] <- model_diff_capm$coefficients[[1]]
        results[j, num_portfolios + 4] <- (
            model_diff_capm$coefficients[[1]] / sqrt(NeweyWest(model_diff_capm, lag = 6)[[1]])
        )
        results[j, num_portfolios + 5] <- model_diff_ff$coefficients[[1]]
        results[j, num_portfolios + 6] <- (
            model_diff_ff$coefficients[[1]] / sqrt(NeweyWest(model_diff_ff, lag = 6)[[1]])
        )}

    
    return (results)}

In [23]:
EW_portfolio_results_size_vol = portfolio_results(EW_portfolio_returns_size_vol, sort_var2 = 'vol')
EW_portfolio_results_size_skew = portfolio_results(EW_portfolio_returns_size_skew, sort_var2 = 'skew')

In [24]:
VW_portfolio_results_size_vol = portfolio_results(VW_portfolio_returns_size_vol, sort_var2 = 'vol')
VW_portfolio_results_size_skew = portfolio_results(VW_portfolio_returns_size_skew, sort_var2 = 'skew')

In [25]:
# function for printing readable amounts of decimals and don't show "NA"
print_custom <- function(df, digits = 5) {
  df[] <- lapply(df, function(x) if(is.numeric(x)) round(x, digits) else x)
  df[is.na(df)] <- ""
  df
}

## c) Fama-MacBeth regression

In the last part of the methodology section, the Fama-MacBeth regression is prepared and estimated. We mainly focus on the effects of volatility and size on future returns. Namely, we estimate separate regressions containing only the volatility/skewness variables, and regressions also controlling for size. The function `perform_fama_macbeth_regression` expects the returns and a list of data containing all the desired explanatory variables.

In [26]:

perform_fama_macbeth_regression <- function (data_explained, data_explanatory) {
    # Initialize an xts object for storing regression results.
    fm_results <- as.xts(
        matrix(
            nrow = nrow(data_explained) - 1,
            ncol = 1 + length(data_explanatory)
        ),
        order.by = index(data_explained[2:nrow(data_explained)])
    )
    column_names <- c('Intercept')
    column_names <- c(column_names, names(data_explanatory))
    names(fm_results) <- column_names

    # Iterate over rows and perform cross-sectional regressions.
    for (i in 1:nrow(fm_results)) {
        current_month <- index(fm_results[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(data_explained)) {
            next
        }

        # Avoid look-ahead bias by selecting the explained variable data from the next month.
        month_data_explained <- data_explained[next_month]
        month_data_explanatory <- c()
        move_to_next_month = FALSE
        for (j in 1:length(data_explanatory)) {
            if (!current_month %in% as.Date(format(index(data_explanatory[[j]])), format="%Y-%m-%d")) {
                move_to_next_month <- TRUE
                break
            }

            colnames(month_data_explanatory)
            cross_section <- t(data_explanatory[[j]][current_month])
            colnames(cross_section) <- names(data_explanatory)[j]

            cross_section[which(cross_section == -Inf)] = NA
            cross_section[which(cross_section == Inf)] = NA

            month_data_explanatory <- cbind(month_data_explanatory, cross_section)
        }

        if (move_to_next_month) {
            next
        }

        transposed_explained <- t(month_data_explained)
        colnames(transposed_explained) <- c('explained')
        equation <- paste('explained ~ 1 + ', paste0(colnames(month_data_explanatory), collapse = ' + '))
        all_data <- na.omit(as.data.frame(cbind(transposed_explained, month_data_explanatory)))

        # Save regression coefficients and other statistics.
        model <- lm(equation, data = all_data)
        for (k in 1:ncol(fm_results)) {
            fm_results[i, k] <- model$coefficients[[k]]
        }
    }

    # Compute time series means, standard errors.
    final_fm_results <- as.data.frame(matrix(nrow = 2, ncol = ncol(fm_results)))
    names(final_fm_results) <- colnames(fm_results)

    for (i in 1:ncol(final_fm_results)) {
        model <- lm(na.omit(fm_results[, i]) ~ 1)
        final_fm_results[1, i] <- model$coefficients[[1]]
        final_fm_results[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6)[[1]])
    }
    rownames(final_fm_results) = c('coefficient estimate', 'NW t-stat')

    final_fm_results
}

# 4. Results

This section presents the results. First, we introduce the theory, and what effects we would expect according to what we have seen during the semester. Next, we go through the actual results of the bivariate sort including a comparison with said theory. Lastly, the Fama-MacBeth regression results are presented, together with an assessment of the economic performance of the variables we have utilized. The focus is primarily on the volatility and skewness factors, but size is also shortly discussed.

According to the theory, the size effect is negative (smaller stocks have on average higher returns). The theory with regards to skewness suggests a possible negative effect of skewness, as investors might require compensation for negatively skewed stocks. We have seen in class that the significance of the results depended on the period used for the data as well as the weighting scheme, sometimes even suggesting a positive skewness effect. The negative effect for total skewness was evident from the Fama-MacBeth regression only after controlling for many other factors. This indicates that the skewness effect might not be as strong and robust, which is something we should keep in mind when evaluating our results.

The empirical evidence of the volatility effect is mixed. Early research suggests a positive relation, while newer studies show a negative puzzling relation. In lectures we have seen this negative puzzling relationship, however, it did also depend on the weighting scheme.

## (a) Bivariate sort results

### (i) Size-skewness

The average one-month ahead excess returns are decreasing with increasing size, regardless of the weighting scheme, for all three skewness portfolios. Also, the CAPM and FF-CPS alphas are significant, meaning that this difference cannot be explained by the factors included in the models. Hence, we observe the negative size effect, in line with the theory.

The wighting scheme in this case does not have a dramatic effect on the results. The size effects as well as the average returns within the portfolio are of similar magnitude in both cases. Hence, it is not the case that the very smallest stocks with very high returns would be driving the results of the equally-weighted portfolio in our data.

As mentioned, during classes we have observed mixed results with respect to the effect of skewness, however, no significant skewness effect is observed here across portfolios of all sizes. It is important to note that we have used a long period for the calculation of the factors, so the results are in line with those of the lectures where the effect was also only statistically significant when the skewness was calculated on a few months of returns data. We have not used skewness for univariate portfolio analysis so it is difficult to say, what is the role of size in these results. But overall, we observe no significant skewness effect when we control for size, the lack of the skewness effect is thus consistent across the different size portfolios, this is also the case for the statistically insignificant alphas.

In [27]:
print("Equally-weighted portfolio")
print("EW: ")
print_custom(EW_portfolio_results_size_skew)

print("Value-weighted portfolio")
print("VW: Betas")
print_custom(VW_portfolio_results_size_skew)

[1] "Equally-weighted portfolio"
[1] "EW: "


,size 1,size 2,size 3,size 3 - 1,size 3 - 1 t-stat,size 3 - 1 CAPM alpha,size 3 - 1 CAPM alpha t-stat,size 3 - 1 FF-CPS alpha,size 3 - 1 FF-CPS alpha t-stat
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
skew 1,-0.05052,-0.05791,-0.06092,-0.01041,-5.92179,-0.01138,-5.11391,-0.01051,-5.44067
skew 2,-0.05328,-0.05989,-0.06174,-0.00847,-4.20561,-0.00882,-4.04022,-0.0085,-3.52465
skew 3,-0.05131,-0.05828,-0.06044,-0.00912,-3.89771,-0.00871,-3.45543,-0.00817,-2.98202
skew 3 - 1,-0.00080,-0.00037,0.00049,,,,,,
skew 3 - 1 t-stat,-0.32798,-0.28341,0.30242,,,,,,
skew 3 - 1 CAPM alpha,-0.00218,-0.00040,0.00049,,,,,,
skew 3 - 1 CAPM alpha t-stat,-0.80847,-0.25878,0.28476,,,,,,
skew 3 - 1 FF-CPS alpha,-0.00216,-0.00013,0.00019,,,,,,
skew 3 - 1 FF-CPS alpha t-stat,-0.89814,-0.08531,0.10990,,,,,,


[1] "Value-weighted portfolio"
[1] "VW: Betas"


,size 1,size 2,size 3,size 3 - 1,size 3 - 1 t-stat,size 3 - 1 CAPM alpha,size 3 - 1 CAPM alpha t-stat,size 3 - 1 FF-CPS alpha,size 3 - 1 FF-CPS alpha t-stat
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
skew 1,-0.04991,-0.05550,-0.05709,-0.00718,-3.83733,-0.00754,-3.54828,-0.00685,-2.94211
skew 2,-0.05297,-0.05742,-0.05857,-0.0056,-2.14323,-0.00584,-2.04555,-0.00529,-1.74791
skew 3,-0.04973,-0.05596,-0.05764,-0.00791,-2.73467,-0.00767,-2.46428,-0.00713,-2.28925
skew 3 - 1,0.00019,-0.00045,-0.00054,,,,,,
skew 3 - 1 t-stat,0.09077,-0.31805,-0.22948,,,,,,
skew 3 - 1 CAPM alpha,-0.00062,-0.00040,-0.00076,,,,,,
skew 3 - 1 CAPM alpha t-stat,-0.29542,-0.24554,-0.27706,,,,,,
skew 3 - 1 FF-CPS alpha,-0.00081,-0.00011,-0.00108,,,,,,
skew 3 - 1 FF-CPS alpha t-stat,-0.41648,-0.06886,-0.40565,,,,,,


### (i) Size-volatility

The size effect remains significant when controlling for volatility instead of skewness, as well as the CAPM alpha and FF-CPS alpha. The weighting scheme does not make a significant difference in the case of the size effect; however, it does seem that it is slightly higher in high-volatility stocks, observed by value of larger magnitude in absolute value in the 3-1 size portfolio of the highest volatility portfolio.

Looking at the long-short portfolio of volatility, we also observe a statistically significant and positive volatility effect in the first-size portfolio for the equally weighted stocks and a positive volatility effect in the first two size portfolios in the value-weighted portfolio. This suggests that we do observe that the returns are on average larger for more volatile stocks, but this only holds for small and moderately large companies, not the largest ones. Looking more at the weighting, we can see that the results are generally very similar and borderline significant (they are significant at the 90% level) even for the equally weighted Size 2 long-short volatility portfolio, hence the effect of the weights is moderate. 

When we contrast the results to what we have seen during the lectures we can note several differences: 1) we observe a positive relation between the volatility and returns. The average excess returns are higher for more volatile stocks 2) The effect exists among smaller stocks. In class we have observed mixed results, some specifications suggested negative relations among large stocks, but such an effect is not observed here.

The significance of the alphas is consistent with the significance of the long-short portfolio for both size and volatility. Both CAPM Alpha and Fama-French CPS model corrections show statistically significant alphas, whenever the '3-1' portfolio is also significant, suggesting that the long-large/short-small (size or volatility) strategy generates abnormal returns over what CAPM and FF CPS-factor models would predict. This could indicate that the strategy successfully captures size and possibly some volatility effects that are not fully explained by these models.

In [28]:
print("Equally-weighted portfolio")
print("EW: ")
print_custom(EW_portfolio_results_size_vol)

print("Value-weighted portfolio")
print("VW: Betas")
print_custom(VW_portfolio_results_size_vol)

[1] "Equally-weighted portfolio"
[1] "EW: "


,size 1,size 2,size 3,size 3 - 1,size 3 - 1 t-stat,size 3 - 1 CAPM alpha,size 3 - 1 CAPM alpha t-stat,size 3 - 1 FF-CPS alpha,size 3 - 1 FF-CPS alpha t-stat
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
vol 1,-0.05572,-0.06167,-0.06014,-0.00442,-3.74914,-0.00453,-3.71185,-0.00437,-3.07849
vol 2,-0.05396,-0.05785,-0.06109,-0.00713,-4.36931,-0.00725,-3.74448,-0.00716,-3.53094
vol 3,-0.04789,-0.05686,-0.06094,-0.01305,-5.88902,-0.01329,-5.14974,-0.01167,-4.21981
vol 3 - 1,0.00783,0.00481,-0.00080,,,,,,
vol 3 - 1 t-stat,2.61528,1.72703,-0.24569,,,,,,
vol 3 - 1 CAPM alpha,0.00890,0.00545,0.00014,,,,,,
vol 3 - 1 CAPM alpha t-stat,2.54852,1.68733,0.03896,,,,,,
vol 3 - 1 FF-CPS alpha,0.00799,0.00540,0.00069,,,,,,
vol 3 - 1 FF-CPS alpha t-stat,2.13389,1.55768,0.19027,,,,,,


[1] "Value-weighted portfolio"
[1] "VW: Betas"


,size 1,size 2,size 3,size 3 - 1,size 3 - 1 t-stat,size 3 - 1 CAPM alpha,size 3 - 1 CAPM alpha t-stat,size 3 - 1 FF-CPS alpha,size 3 - 1 FF-CPS alpha t-stat
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
vol 1,-0.05512,-0.06023,-0.05754,-0.00243,-1.71594,-0.00266,-1.91668,-0.00249,-1.61851
vol 2,-0.05360,-0.05598,-0.05748,-0.00388,-1.62935,-0.00355,-1.40895,-0.00345,-1.26951
vol 3,-0.04615,-0.05303,-0.05734,-0.01119,-3.37954,-0.01147,-3.16515,-0.0099,-2.74782
vol 3 - 1,0.00897,0.00719,0.00021,,,,,,
vol 3 - 1 t-stat,2.82789,2.56621,0.05457,,,,,,
vol 3 - 1 CAPM alpha,0.00993,0.00801,0.00112,,,,,,
vol 3 - 1 CAPM alpha t-stat,2.78758,2.44340,0.27206,,,,,,
vol 3 - 1 FF-CPS alpha,0.00930,0.00800,0.00189,,,,,,
vol 3 - 1 FF-CPS alpha t-stat,2.45274,2.32292,0.48785,,,,,,


## (b) Fama-MacBeth regression results

This section presents the results of the Fama-MacBeth regression. The effects are studied both independently and together. The results are discussed here, and are presented below. 

We do not observe statistically significant cross-sectional relation between skewness and $r_{t+1}$. We thus do not calculate economic significance for skewness, since the effect is not statistically different from zero. 

On the other hand, the cross-sectional relation between volatility and future stock returns is positive and statistically significant across specifications. The economic significance is calculated from the final regression containing all three regressors; however, the point estimate for the volatility variable coefficient is around 0.001 in all the regressions.

Everything else equal, one standard deviation difference in volatility is associated with a $0.00104 \times 2.4629 = 0.0026\%$ per month change in expected returns. This means around 0.03% change per year. The relationship is thus statistically significant, yet not very economically important and definitely much less than what we have seen in other factors during the semester, such as momentum or value.

The results are consistent across specifications and do not change significantly with more factors added. Generally, our analysis suggests that  size negatively correlates with returns, there is a positive relation with volatility, and skewness does not play a significant role. 

In [29]:
# SD to determine economic importance of volatility
mean_monthly_sd_vol

[1] 2.462899

### (i) Skewness

In [30]:
data_explanatory = list(monthly_skewness)
names(data_explanatory) = c("skew")
fm_skew = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_skew)

,Intercept,skew
,<dbl>,<dbl>
coefficient estimate,-0.05750,0.00023
NW t-stat,-1.92045,0.26437


### (ii) Size-Skewness

Similarly to the bivariate sort, we do not find a statistically significant effect of skewness using the Fama-MacBeth regression approach.

In [31]:
data_explanatory = list(monthly_sizes, monthly_skewness)
names(data_explanatory) = c("size", "skew")
fm_size_skew = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_size_skew)

,Intercept,size,skew
,<dbl>,<dbl>,<dbl>
coefficient estimate,-0.03868,-0.00318,0.00007
NW t-stat,-1.42692,-6.18217,0.08438


### (iii) volatility

In [32]:
data_explanatory = list(monthly_volatilities)
names(data_explanatory) = c("volatility")
fm_vol = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_vol)

,Intercept,volatility
,<dbl>,<dbl>
coefficient estimate,-0.06565,0.00121
NW t-stat,-1.73712,2.63451


### (iv) Size-volatility

In [33]:
data_explanatory = list(monthly_sizes, monthly_volatilities)
names(data_explanatory) = c("size", "volatility")
fm_size_vol = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_size_vol)

,Intercept,size,volatility
,<dbl>,<dbl>,<dbl>
coefficient estimate,-0.04994,-0.00250,0.00103
NW t-stat,-1.62475,-4.95998,2.22596


### (v) Skewness-volatility

In [34]:
data_explanatory = list(monthly_volatilities, monthly_skewness)
names(data_explanatory) = c("volatility", "skew")
fm_vol_skew = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_vol_skew)

,Intercept,volatility,skew
,<dbl>,<dbl>,<dbl>
coefficient estimate,-0.06570,0.00123,0.00014
NW t-stat,-1.73486,2.66232,0.22854


### (vi) all three variables 

In [35]:
data_explanatory = list(monthly_sizes, monthly_volatilities, monthly_skewness)
names(data_explanatory) = c("size", "volatility", "skew")
fm_size_vol_skew = perform_fama_macbeth_regression(monthly_returns, data_explanatory)
print_custom(fm_size_vol_skew)

,Intercept,size,volatility,skew
,<dbl>,<dbl>,<dbl>,<dbl>
coefficient estimate,-0.05017,-0.00247,0.00104,0.00001
NW t-stat,-1.63266,-4.85405,2.24806,0.01609


# 4. Conclusion

We examine the cross-sectional relation between future stock returns and sort variables: size, skewness, and volatility, with a focus on the latter two. We use the bivariate independent sort analysis, creating 3x3 portfolios as well as estimating a Fama-MacBeth regression with the variables of interest.

Our findings confirmed a statistically significant, though economically modest, positive correlation between volatility and future returns. Both the CAPM and Fama-French alphas remained significant in the long-short volatility portfolios in the bivariate sort, suggesting that the factors in those models cannot explain the relationship. Moreover, the bivariate sort revealed that the positive effect of volatility is not present in the largest stocks, as the difference portfolio did not show significant results within the last size portfolio. 

Contrary to volatility, skewness did not show a statistically significant relationship. Consequently, we confirm that the relationship is not as strong as that of the other variables. We have seen in class that the significant relationship might only appear after controlling for many other factors that we haven't used in the Fama-MacBeth regression. This is thus something that could be explored in the future.